<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/get_posts_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script processes text data, tokenizes it and saves it to a CSV file, optionally taking into account the posts' affiliation to a specific VK group.

Этот скрипт обрабатывает текстовые данные, токенизирует их и сохраняет в CSV-файл опционально с учетом принадлежности постов к конкретной ВК группе.

In [3]:
!pip install -U pymorphy3
!pip install emoji
import pandas as pd
import requests
import csv
from io import StringIO
filename = 'text_preprocessing.py'
response = requests.get(f'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/src/vk/nlp/{filename}')
with open(filename,'w+') as f:
  f.write(response.text)
import text_preprocessing

In [4]:
# domains = ['club221681617','concerto','club151359929','pravoslav_karelia']# smallest groups for tests
domains = ['aparfenchikov','minnazrk', 'mincultrk', 'uoknrk']# state_tokens.csv

#folder = 'religion'
folder = ''
url = f'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/{folder}'

def tokens_of_posts(domains, url, flag_id_group=False):
  if flag_id_group:
    df = pd.concat([pd.read_csv(
            f'{url}/{domain}.csv',
            usecols=['text', 'date']
        ).assign(group_id=domain)
        for domain in domains
    ], ignore_index=True).sort_values('date')

  else:
    df = pd.concat([pd.read_csv(
            f'{url}/{domain}.csv',
            usecols = ['text','date'])
           for domain in domains],
          ignore_index=True).sort_values('date')
  df = df[df['text'].notna() & df['text'].astype(str).str.strip().astype(bool)].reset_index(drop=True)

  """
  Используется функция process_text() из модуля text_preprocessing, которая принимает на вход два параметра:
  text - предложения в формате строки для обработки
  points - параметр по умолчанию, чтобы получить токенизацию формата, необходимого для Voyants Tools, нужно вызвать функция с явным указанием параметра в значении True
  Тогда будет сгененрирован файл с точечной периодизацией
  Пример для Voyant Tools: process_text(text, True)
  Пример для тематического моделирования process_text(text)

  """
  df['tokens'] = df['text'].apply(text_preprocessing.process_text)
  # Removing lines with empty 'tokens'
  df = df[df['tokens'].str.strip().astype(bool)]

  # Save CSV with quotes only for 'tokens' field, without quotes for 'date'
  with open('tokens_with_group_id.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_NONE, quotechar=None)
    if flag_id_group:
      writer.writerow(['tokens', 'date','group_id'])
      for _, row in df.iterrows():
        writer.writerow([f'"{row["tokens"]}"', row['date'], f'"{row["group_id"]}"'])
    else:
      writer.writerow(['tokens', 'date'])
      for _, row in df.iterrows():
        writer.writerow([f'"{row["tokens"]}"', row['date']])
  return df

"""
  Используется функция tokens_of_posts(), которая принимает на вход три параметра:
  domains - id групп в формате списка строк
  url - ссылка расположения файла с постами
  flag_id_group - параметр по умолчанию, чтобы получить файл со столбцом 'group_id, необходимо при вызове явно указать параметр в значении True
  Пример для генерации файла с колонкой group_id: tokens_of_posts(domains,url,True)
"""
df = tokens_of_posts(domains,url)
